In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from collections import defaultdict

In [4]:
from kp.utils.constants import EXPERIMENTS_DIR
from kp.plotting.plotting import find_results_files, organize_results, plot_metric

In [4]:
base_experiments_dir = EXPERIMENTS_DIR
layers_setting = "lt_reversal_baseline_top_quarter"
# layers_setting = "all_layers"
base_experiments_dir = base_experiments_dir / layers_setting
base_experiments_dir

PosixPath('/net/projects/clab/tnief/bidirectional-reversal/experiments/lt_reversal_baseline')

In [5]:
allow_smoke_test = False
all_my_results_files = find_results_files(base_experiments_dir, allow_smoke_test)

Found 30 'results.json' files.


In [6]:
all_my_results_files

[PosixPath('/net/projects/clab/tnief/bidirectional-reversal/experiments/lt_reversal_baseline/lm_head_never/fake_movies_real_actors/gpt2-xl/sft2pre/attn_ffn/all_2025-05-07_21-56-24_best_saved_checkpoint_2025-05-14_22-25-41/sentence_1/dropout_0.0_layer_count/results.json'),
 PosixPath('/net/projects/clab/tnief/bidirectional-reversal/experiments/lt_reversal_baseline/lm_head_never/fake_movies_real_actors/gpt2-xl/sft2pre/o_ffn/all_2025-05-07_21-56-24_best_saved_checkpoint_2025-05-14_22-25-41/sentence_1/dropout_0.0_layer_count/results.json'),
 PosixPath('/net/projects/clab/tnief/bidirectional-reversal/experiments/lt_reversal_baseline/lm_head_never/fake_movies_real_actors/gpt2-xl/sft2pre/o/all_2025-05-07_21-56-24_best_saved_checkpoint_2025-05-14_22-25-41/sentence_1/dropout_0.0_layer_count/results.json'),
 PosixPath('/net/projects/clab/tnief/bidirectional-reversal/experiments/lt_reversal_baseline/lm_head_never/fake_movies_real_actors/gpt2-xl/sft2pre/ffn/all_2025-05-07_21-56-24_best_saved_check

In [7]:
organized_results = organize_results(all_my_results_files, base_experiments_dir)

Attempted to parse 30 files.
Successfully parsed 30 paths and calculated metrics for 30.
Organized data into 1 datasets.


In [8]:
organized_results

defaultdict(<function kp.plotting.plotting.organize_results.<locals>.<lambda>()>,
            {'fake_movies_real_actors': defaultdict(<function kp.plotting.plotting.organize_results.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'lm_head_never': defaultdict(<function kp.plotting.plotting.organize_results.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                                      {'gpt2-xl': defaultdict(dict,
                                                   {'sentence_1': {'attn_ffn': {'mean_target_rank': 353.06606606606607,
                                                      'top_k_accuracy': 0.008008008008008008,
                                                      'mean_target_prob': 0.0011879963780660906},
                                                     'o_ffn': {'mean_target_rank': 350.005005005005,
                                                      'top_k_accuracy': 0.008008008008008008,
                                          

In [9]:
lm_head_setting = "lm_head_never"
filtered_data = {
    dataset: {
        lm_head_setting: models[lm_head_setting]
    }
    for dataset, models in organized_results.items()
    if lm_head_setting in models
}

In [10]:
filtered_data

{'fake_movies_real_actors': {'lm_head_never': defaultdict(<function kp.plotting.plotting.organize_results.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
              {'gpt2-xl': defaultdict(dict,
                           {'sentence_1': {'attn_ffn': {'mean_target_rank': 353.06606606606607,
                              'top_k_accuracy': 0.008008008008008008,
                              'mean_target_prob': 0.0011879963780660906},
                             'o_ffn': {'mean_target_rank': 350.005005005005,
                              'top_k_accuracy': 0.008008008008008008,
                              'mean_target_prob': 0.001196711189049392},
                             'o': {'mean_target_rank': 543.3403403403404,
                              'top_k_accuracy': 0.013013013013013013,
                              'mean_target_prob': 0.001625634856729327},
                             'ffn': {'mean_target_rank': 360.1041041041041,
                              'top_k_ac

In [11]:
def filter_by_sentence(data, sentence):
    # Deeply traverse the nested defaultdict structure
    filtered_data = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))
    
    for dataset, models in data.items():
        for model_type, model_dict in models.items():
            for model_name, sentence_dict in model_dict.items():
                if sentence in sentence_dict:
                    filtered_data[dataset][model_type][model_name][sentence] = sentence_dict[sentence]
    
    return filtered_data


# Usage:
sentence = "sentence_1"
filtered_data = filter_by_sentence(filtered_data, sentence)

In [1]:
metric_key = "top_k_accuracy"
core_patches_only = False
short_title = True
save = True
plot_metric(filtered_data, metric_key, layers_setting=layers_setting, save=save, core_patches_only=core_patches_only, short_title=short_title)

NameError: name 'plot_metric' is not defined

In [ ]:
metric_key = "mean_target_rank"
plot_metric(organized_results, metric_key)

In [ ]:
metric_key = "mean_target_prob"
plot_metric(organized_results, metric_key)